In [ ]:
import data_reader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
path2submits  = 'iui20_ideaSubmitsPart2.csv'
path2requests = 'iui20_inspirationRequestsPart2.csv'
seconds_per_bucket = 60
train_percentage = 0.7

In [ ]:
#load data as sessions
sessions = data_reader.load_data_as_sessions_dict(path2submits, path2requests, seconds_per_bucket)
# calculate avoiders and seekers
sessions = data_reader.add_avoiders_and_seekers(sessions)
# create train and test set of worker ids
train_worker_ids, test_worker_ids = data_reader.split_worker_ids_into_train_test(sessions, train_percentage=train_percentage)

In [ ]:
# iterate through different numbers of buckets
# train a classifier for each different numbers of buckets
# test on test examples
# get roc curve
conf_matrices = []
max_number_of_buckets = 15
for nr_of_buckets in range(1, max_number_of_buckets):
    x_tr, y_tr, x_te, y_te = data_reader.create_train_test_dataset(nr_of_buckets, train_worker_ids, test_worker_ids, sessions, train_percentage)
    #clf = DecisionTreeClassifier()
    clf = RandomForestClassifier(200)
    clf.fit(x_tr, y_tr)
    y_pre = clf.predict(x_te)
    conf_matrices.append(confusion_matrix(y_te, y_pre))

# [[tn, fp, fn, tp], ...]
conf_matrices = np.array([conf_m.ravel() for conf_m in conf_matrices])
accuracies = [(cm[0]+cm[3])/sum(cm) for cm in conf_matrices]
precisions = [cm[3]/(cm[3]+cm[1])   for cm in conf_matrices]
recall     = [cm[3]/(cm[3]+cm[2])   for cm in conf_matrices]

print(conf_matrices)

plt.plot(accuracies, label='accuracy')
plt.plot(precisions, label='precision')
plt.plot(recall,     label='recall')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=3, mode="expand", borderaxespad=0.)
plt.show()

In [ ]:
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

train_percentage = 1.0
nr_of_buckets = 10
x_tr, y_tr, x_te, y_te = data_reader.create_train_test_dataset(nr_of_buckets, train_worker_ids, test_worker_ids, sessions, train_percentage)
clf = DecisionTreeClassifier()
clf.fit(x_tr, y_tr)
    
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data, filled=True, rounded=True, special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())